In [14]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [15]:
lam = 6 # Streckung
    
b = 15 # m Spannweite
c = b/lam # Flügeltiefe
y_li = [0,b/2] # Stüzstellen
c_li = [c]*2
alpha = 1 # Grad

dcl = 2*np.pi
N_M = 7

In [41]:
# calculate wingspan
b = 2*max(y_li)

# calculate wing area
S = 2 * np.trapz(y=c_li, x=y_li)

# calculate aspect ratio
AR = b ** 2 / S

# number of grid points
if N_M is None:
    N_M = int(round(AR)*4-1)  # has to be uneven, not more than 4*aspect ratio

# grid point indices
v_ar = np.arange(1,N_M)

# create thetas
theta_ar = v_ar * np.pi / (N_M + 1)

# array of grid points
y_ar = b/2 * np.cos(theta_ar)

# chord depthes
chord_ar = np.interp(np.abs(y_ar), y_li, c_li)

# distribute lift slope over wing
if not isinstance(dcl,np.ndarray):
    dcl *= np.ones(N_M)
elif len(dcl) != N_M:
    dcl = np.interp(y_ar,y_li,dcl)

# distribute aoa over wing
if not isinstance(alpha, np.ndarray) or len(alpha)==1:
    alpha = np.ones(N_M)*alpha
elif len(alpha) == len(y_li):
    # interpolate
    alpha = np.interp(y_ar, y_li, alpha, left=alpha[0], right=0)
else:
    print('error') #TODO error handling

#v_ar = np.arange(1, N_M + 1)

# calculate thetas
theta_ar = np.array(np.linspace(0,np.pi,N_M))

# create empyt matrix (N_MxN_M) for multhoppcoefficients
B = np.zeros((N_M, N_M))

# calculation of multhopp coefficients
for v,y_v,theta_v,c,dcl_v in zip(v_ar,y_ar,theta_ar,chord_ar,dcl):
    for n,theta_n in zip(v_ar,theta_ar):

        # diagonal elements
        if(v==n):
            B[v-1,v-1]= (N_M+1)/(4*np.sin(theta_v))+2*b/(dcl_v*c)
        # non diagonal elements
        else:
            B[v-1,n-1]=-((1-(-1.)**(v-n))/2*(np.sin(theta_n)/((N_M+1)*(np.cos(theta_n)-np.cos(theta_v))**2)))

# calculation of local circulation
gamma_ar = np.dot(np.linalg.inv(B),alpha)

# lift coefficient for whole wing
C_A = np.pi*AR/(N_M+1)*sum(gamma_ar*np.sin(v_ar*np.pi/(N_M+1)))

# induced angle
a_ind = np.zeros(N_M)

for v in range(N_M):
    part1 = (N_M+1)/(4*np.sin((v+1)*np.pi/(N_M+1)))*gamma_ar[v]

    part2 = 0

    for j in range(N_M):
        if j == v:
            continue
        part2 += B[v,j]*gamma_ar[j]

    a_ind[v] = part1 + part2 # should be a subtraction, but only works with addition

# calculate induced drag
C_Wi = np.pi*AR/(N_M+1) * sum(gamma_ar*a_ind*np.sin(v_ar*np.pi/(N_M+1)))

# Oswald-factor
k = C_A**2/(np.pi*AR*C_Wi)

# local lift coefficient
c_a_li = 2*b/(chord_ar)*gamma_ar

# calculate effective aoa
a_eff = alpha-a_ind


error


/home/jonathan/.anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: divide by zero encountered in double_scalars


LinAlgError: Singular matrix

In [42]:
v_ar

array([1, 2, 3, 4, 5, 6])

In [40]:
np.cos()

array([ 1.00000000e+00,  8.66025404e-01,  5.00000000e-01,  6.12323400e-17,
       -5.00000000e-01, -8.66025404e-01, -1.00000000e+00])